In [ ]:
import polars as pl

In [ ]:
!gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_03.parquet /home/eanegrin/datasets/

In [ ]:
base_path = '/home/eanegrin/'
# base_path = 'C:/Eugenio/Maestria/DMEyF/'

dataset_path = base_path + 'datasets/'

dataset_file = 'competencia_03.parquet'

In [ ]:
df = pl.read_parquet(dataset_path + dataset_file)

In [ ]:
columnas = [
    "mrentabilidad"
    , "mrentabilidad_annual"
    , "mcomisiones"
    , "mactivos_margen"
    , "mpasivos_margen"
    , "mcuenta_corriente_adicional"
    , "mcuenta_corriente"
    , "mcaja_ahorro"
    , "mcaja_ahorro_adicional"
    , "mcaja_ahorro_dolares"
    , "mcuentas_saldo"
    , "mautoservicio"
    , "mtarjeta_visa_consumo"
    , "mtarjeta_master_consumo"
    , "mprestamos_personales"
    , "mprestamos_prendarios"
    , "mprestamos_hipotecarios"
    , "mplazo_fijo_dolares"
    , "mplazo_fijo_pesos"
    , "minversion1_pesos"
    , "minversion1_dolares"
    , "minversion2"
    , "mpayroll"
    , "mpayroll2"
    , "mcuenta_debitos_automaticos"
    , "mttarjeta_visa_debitos_automaticos"
    , "mttarjeta_master_debitos_automaticos"
    , "mpagodeservicios"
    , "mpagomiscuentas"
    , "mcajeros_propios_descuentos"
    , "mtarjeta_visa_descuentos"
    , "mtarjeta_master_descuentos"
    , "mcomisiones_mantenimiento"
    , "mcomisiones_otras"
    , "mforex_buy"
    , "mforex_sell"
    , "mtransferencias_recibidas"
    , "mtransferencias_emitidas"
    , "mextraccion_autoservicio"
    , "mcheques_depositados"
    , "mcheques_emitidos"
    , "mcheques_depositados_rechazados"
    , "mcheques_emitidos_rechazados"
    , "matm"
    , "matm_other"
    , "Master_mfinanciacion_limite"
    , "Master_msaldototal"
    , "Master_msaldopesos"
    , "Master_msaldodolares"
    , "Master_mconsumospesos"
    , "Master_mconsumosdolares"
    , "Master_mlimitecompra"
    , "Master_madelantopesos"
    , "Master_madelantodolares"
    , "Master_mpagado"
    , "Master_mpagospesos"
    , "Master_mpagosdolares"
    , "Master_mconsumototal"
    , "Master_mpagominimo"
    , "Visa_mfinanciacion_limite"
    , "Visa_msaldototal"
    , "Visa_msaldopesos"
    , "Visa_msaldodolares"
    , "Visa_mconsumospesos"
    , "Visa_mconsumosdolares"
    , "Visa_mlimitecompra"
    , "Visa_madelantopesos"
    , "Visa_madelantodolares"
    , "Visa_mpagado"
    , "Visa_mpagospesos"
    , "Visa_mpagosdolares"
    , "Visa_mconsumototal"
    , "Visa_mpagominimo" 
]

In [ ]:
df = df.with_columns([
    pl.col(col)
    .shift(1)
    .over("numero_de_cliente")
    .sort_by(pl.col("foto_mes"))  
    .alias(f"{col}_lag1")
    for col in columnas
])

In [ ]:
df.select([
    pl.col("foto_mes"),
    pl.col("mrentabilidad"),
    pl.col("mrentabilidad_lag1")
]).head(20)